In [2]:
from preprocessing import *
from model_custom_roberta import *

import argparse
import pandas as pd
import json

from transformers import RobertaConfig, get_scheduler
from roberta_config import CustomRobertaConfig
from accelerate import Accelerator, DistributedDataParallelKwargs
from torch.optim import AdamW
from datasets import load_dataset, load_metric
import copy

import dotenv
dotenv.load_dotenv()


/home/lvpoellhuber/env/bmir-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/lvpoellhuber/storage


True

In [3]:
roberta_config_dict = {
        "max_position_embeddings": 514,
        "hidden_size": 768,
        "num_attention_heads": 12,
        "num_hidden_layers": 6,
        "type_vocab_size": 1,
        "attn_mechanism": "eager",
        "num_labels": 2
    }

adaptive_config_dict = {
        "max_position_embeddings": 514,
        "hidden_size": 768,
        "num_attention_heads": 12,
        "num_hidden_layers": 6,
        "type_vocab_size": 1,
        "attn_mechanism": "adaptive",
        "num_labels":4,
        "inner_hidden_size": 1024,
        "dropout": 0,
        "attn_span": 1024,
        "adapt_span_enabled": True,
        "adapt_span_loss": 2e-06,
        "adapt_span_ramp": 32,
        "adapt_span_init": 0,
        "adapt_span_cache": True
    }

sparse_config_dict = {
        "vocab_size": 32,
        "max_position_embeddings": 512,
        "hidden_size": 768,
        "num_attention_heads": 12,
        "num_hidden_layers": 6,
        "type_vocab_size": 1,
        "attn_mechanism": "sparse"
    }

In [ ]:
tokenizer = get_tokenizer("FacebookAI/roberta-base")

roberta_config = CustomRobertaConfig.from_dict(roberta_config_dict)
roberta_config.vocab_size = tokenizer.vocab_size
roberta_config.num_labels = 2

adaptive_config = CustomRobertaConfig.from_dict(adaptive_config_dict)
adaptive_config.vocab_size = tokenizer.vocab_size
adaptive_config.num_labels = 2

sparse_config = CustomRobertaConfig.from_dict(sparse_config_dict)
sparse_config.vocab_size = tokenizer.vocab_size
sparse_config.num_labels = 2

In [ ]:
roberta_model = RobertaForSequenceClassification(config=roberta_config).from_pretrained("FacebookAI/roberta-base", config=roberta_config, ignore_mismatched_sizes=True)
adaptive_model = RobertaForSequenceClassification(config=adaptive_config).from_pretrained("FacebookAI/roberta-base", config=adaptive_config, ignore_mismatched_sizes=True)
sparse_model = RobertaForSequenceClassification(config=sparse_config).from_pretrained("FacebookAI/roberta-base", config=sparse_config, ignore_mismatched_sizes=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'roberta.encoder.layer.0.attention.self.seq_attention.adaptive_span._mask.current_val', 'roberta.encoder.layer.0.attention.self.seq_attention.adaptive_span._mask.mask_template', 'roberta.encoder.layer.0.attention.self.seq_attention.mask.current_val', 'roberta.encoder.layer.0.attention.self.seq_attention.mask.mask_template', 'roberta.encoder.layer.1.attention.self.s

In [13]:
for i, param in enumerate(roberta_model.named_parameters()):
    if not param[1].requires_grad:
        print(f"Parameter {param[0]} does not require gradient. ")

for i, param in enumerate(adaptive_model.named_parameters()):
    if not param[1].requires_grad:
        print(f"Parameter {param[0]} does not require gradient. ")
        
for i, param in enumerate(sparse_model.named_parameters()):
    if not param[1].requires_grad:
        print(f"Parameter {param[0]} does not require gradient. ")
i

110

In [32]:
test = ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'roberta.encoder.layer.0.attention.self.seq_attention.adaptive_span._mask.current_val', 'roberta.encoder.layer.0.attention.self.seq_attention.adaptive_span._mask.mask_template', 'roberta.encoder.layer.0.attention.self.seq_attention.mask.current_val', 'roberta.encoder.layer.0.attention.self.seq_attention.mask.mask_template', 'roberta.encoder.layer.1.attention.self.seq_attention.adaptive_span._mask.current_val', 'roberta.encoder.layer.1.attention.self.seq_attention.adaptive_span._mask.mask_template', 'roberta.encoder.layer.1.attention.self.seq_attention.mask.current_val', 'roberta.encoder.layer.1.attention.self.seq_attention.mask.mask_template', 'roberta.encoder.layer.2.attention.self.seq_attention.adaptive_span._mask.current_val', 'roberta.encoder.layer.2.attention.self.seq_attention.adaptive_span._mask.mask_template', 'roberta.encoder.layer.2.attention.self.seq_attention.mask.current_val', 'roberta.encoder.layer.2.attention.self.seq_attention.mask.mask_template', 'roberta.encoder.layer.3.attention.self.seq_attention.adaptive_span._mask.current_val', 'roberta.encoder.layer.3.attention.self.seq_attention.adaptive_span._mask.mask_template', 'roberta.encoder.layer.3.attention.self.seq_attention.mask.current_val', 'roberta.encoder.layer.3.attention.self.seq_attention.mask.mask_template', 'roberta.encoder.layer.4.attention.self.seq_attention.adaptive_span._mask.current_val', 'roberta.encoder.layer.4.attention.self.seq_attention.adaptive_span._mask.mask_template', 'roberta.encoder.layer.4.attention.self.seq_attention.mask.current_val', 'roberta.encoder.layer.4.attention.self.seq_attention.mask.mask_template', 'roberta.encoder.layer.5.attention.self.seq_attention.adaptive_span._mask.current_val', 'roberta.encoder.layer.5.attention.self.seq_attention.adaptive_span._mask.mask_template', 'roberta.encoder.layer.5.attention.self.seq_attention.mask.current_val', 'roberta.encoder.layer.5.attention.self.seq_attention.mask.mask_template']

len(test)

28